In [1]:
import sys
sys.path.append('../py_scripts')

In [2]:
import keras
import os
from model_utils import get_mobilenet_model
from utils import get_train_test_data_dict, DictImageDataGenerator, preprocess_func

# define train params
IMG_SIZE = 384
LABEL_CNT = 5004
ALL_DATA_JSON = '../data/train_data.json'
BATCH_SIZE = 16
print('load done')


Using TensorFlow backend.


load done


In [3]:
# get generator
train_d, val_d = get_train_test_data_dict(ALL_DATA_JSON, 0.2, use_new_whale=False, duplicate_low_cls=False)

# simple check
cnt = 0
for k in val_d:
    if len(val_d[k])>0:
        cnt += 1
print('val cls cnt',cnt)

train_ds = DictImageDataGenerator(rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  preprocessing_function=preprocess_func)
train_gen = train_ds.flow_from_dict(train_d, target_size=(
    IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE)
val_ds = DictImageDataGenerator(preprocessing_function=preprocess_func)
val_gen = val_ds.flow_from_dict(val_d, target_size=(
    IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE)
val_steps = val_gen.samples//BATCH_SIZE
print(val_steps)
# test
for x, y in train_gen:
    print(x.shape, y.shape)
    break

val cls cnt 805
Found 14284 images belonging to 5004 classes.
Using bbox for image
Found 1413 images belonging to 5004 classes.
Using bbox for image
88
(16, 384, 384, 3) (16, 5004)


In [4]:
# get baseline mobilenet model
mob_model = get_mobilenet_model(IMG_SIZE, LABEL_CNT, dense_dim=None)
mob_model.summary()

/home/jac/.local/lib/python3.5/site-packages/keras_applications/mobilenet.py:206: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 384, 384, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 385, 385, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 192, 192, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 192, 192, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 192, 192, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 192, 192, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 192, 192, 32)      128       
__________

In [ ]:
# compile
def get_callbacks(model_save_path, model=None):
    if os.path.exists(model_save_path):
        if model is not None:
            model.load_weights(model_save_path, by_name=True, skip_mismatch=True)
            print('load weights')
    model_dir = os.path.dirname(model_save_path)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    m_reduce = keras.callbacks.ReduceLROnPlateau(
        monitor='loss', factor=0.5, patience=3, verbose=1, min_lr=0.00000001)
    m_check = keras.callbacks.ModelCheckpoint(
        model_save_path, monitor='val_loss', verbose=1, save_best_only=True)
    return [m_reduce, m_check]

cb_list = get_callbacks('../data/checkpoints/mob_384_no_new_bbox.h5', mob_model)
adam_opt = keras.optimizers.Adam(lr=0.001)
mob_model.compile(optimizer=adam_opt,
                  loss='categorical_crossentropy', 
                  metrics=['acc'])
print('compile done')

compile done


In [ ]:
# train
mob_model.fit_generator(
    train_gen,
    steps_per_epoch=1000,
    epochs=100,
    verbose=1,
    callbacks=cb_list,
    validation_data=val_gen,
    validation_steps=val_steps
)

Epoch 1/100
  60/1000 [>.............................] - ETA: 18:06 - loss: 8.6631 - acc: 0.0042